In [1]:
from theano.sandbox import cuda

In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras import optimizers

Using Theano backend.


In [3]:
model_path = '/Users/mdymshits/fastai/data/imdb/models/'

In [ ]:
# %mkdir -p /Users/mdymshits/fastai/data/imdb/models/

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list:

In [5]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [6]:
idx2word = {v: k for k, v in idx.iteritems()}

We download the reviews using code copied from keras.datasets:

In [7]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [8]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Pad (with zero) or truncate each sentence to make consistent length.

In [9]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

### Single conv layer 

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [14]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
#     Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
#     Dropout(0.2),
#     MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
#     Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [15]:
sgd = optimizers.SGD(lr=0.01)
conv1.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

```
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 4s - loss: 0.4984 - acc: 0.7250 - val_loss: 0.2922 - val_acc: 0.8816
Epoch 2/4
25000/25000 [==============================] - 4s - loss: 0.2971 - acc: 0.8836 - val_loss: 0.2681 - val_acc: 0.8911
Epoch 3/4
25000/25000 [==============================] - 4s - loss: 0.2568 - acc: 0.8983 - val_loss: 0.2551 - val_acc: 0.8947
Epoch 4/4
25000/25000 [==============================] - 4s - loss: 0.2427 - acc: 0.9029 - val_loss: 0.2558 - val_acc: 0.8947
```

In [16]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=10, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 83s - loss: 0.6932 - acc: 0.5005 - val_loss: 0.6931 - val_acc: 0.5001
Epoch 2/10
25000/25000 [==============================] - 82s - loss: 0.6931 - acc: 0.5047 - val_loss: 0.6931 - val_acc: 0.5104
Epoch 3/10
25000/25000 [==============================] - 83s - loss: 0.6931 - acc: 0.5080 - val_loss: 0.6931 - val_acc: 0.5174
Epoch 4/10
25000/25000 [==============================] - 78s - loss: 0.6930 - acc: 0.5068 - val_loss: 0.6931 - val_acc: 0.5031
Epoch 5/10
25000/25000 [==============================] - 74s - loss: 0.6930 - acc: 0.5143 - val_loss: 0.6930 - val_acc: 0.5161
Epoch 6/10
25000/25000 [==============================] - 75s - loss: 0.6929 - acc: 0.5150 - val_loss: 0.6930 - val_acc: 0.5086
Epoch 7/10
25000/25000 [==============================] - 71s - loss: 0.6928 - acc: 0.5141 - val_loss: 0.6930 - val_acc: 0.5012
Epoch 8/10
25000/25000 [==============================

That's well past the Stanford paper's accuracy - another win for CNNs!

In [ ]:
% ls -1 /Users/mdymshits/fastai/data/

In [29]:
conv1.save_weights(model_path + 'conv1.h5')

In [ ]:
%ls -1 /Users/mdymshits/fastai/data/imdb/models/

In [ ]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [30]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [ ]:
%ls -1 /Users/mdymshits/fastai/data/glove/results/

In [31]:
vecs, words, wordidx = load_vectors('/Users/mdymshits/fastai/data/glove/results/6B.50d')

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [32]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))
    counter = 0
    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            counter += 1
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb, counter

In [33]:
emb, c = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [ ]:
emb.shape

In [62]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])
#     Dense(2, activation ='softmax')])

In [63]:
keras.__version__

'1.2.2'

In [64]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [65]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 1128s - loss: 0.5893 - acc: 0.6788 - val_loss: 0.4978 - val_acc: 0.7877
Epoch 2/5
25000/25000 [==============================] - 177s - loss: 0.5051 - acc: 0.7625 - val_loss: 0.4613 - val_acc: 0.8121
Epoch 3/5
25000/25000 [==============================] - 172s - loss: 0.4751 - acc: 0.7778 - val_loss: 0.4491 - val_acc: 0.8056
Epoch 4/5
25000/25000 [==============================] - 177s - loss: 0.4529 - acc: 0.7892 - val_loss: 0.4247 - val_acc: 0.8251
Epoch 5/5
25000/25000 [==============================] - 168s - loss: 0.4432 - acc: 0.7948 - val_loss: 0.4664 - val_acc: 0.7728


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [55]:
model.layers[0].trainable=True

In [56]:
model.optimizer.lr=1e-4

In [57]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 157s - loss: 0.4683 - acc: 0.7788 - val_loss: 0.4657 - val_acc: 0.7770


As expected, that's given us a nice little boost. :)

In [ ]:
model.save_weights(model_path+'glove50.h5')

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [ ]:
from keras.layers import Merge

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [ ]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out) 

In [ ]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [ ]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable, and then set it to non-trainable after a couple of epochs. I have no idea why!

In [ ]:
model.layers[0].trainable=False

In [ ]:
model.optimizer.lr=1e-5

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

This more complex architecture has given us another boost in accuracy.

## LSTM

We haven't covered this bit yet!

In [ ]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100, consume_less='gpu'),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)